In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
import time,copy
from sklearn.model_selection import train_test_split
import sys
sys.path.append("../Libs")
from Repres_utils import bm_to_graph,find_path,find_all_paths,distmat,append_dict,build_i_idx,get_dihedral,\
                        dihedral_cos,angle_cos,mol_integrity,ordered_charges
from parallel_representations import multi_process_repr
from Representations import build_bond_repr,build_angle_repr,build_dihedral_repr
from sklearn.ensemble import RandomForestRegressor
import pickle



In [ ]:
data = np.load("../Read Data/DataSet_H_IC.npz", allow_pickle=True)
X_IC,Y_IC  = data["x"], data["y"]
Dat=np.vstack((X_IC.T,Y_IC)).T
Dat.shape

In [ ]:
mols=multi_process_repr(Dat[:],"dihedrals",num_processes = 35)

In [ ]:
train,test=train_test_split(mols)

In [ ]:
train_lin={}
train_ring={}
test_lin={}
test_ring={}
for mol in train:
    for d in mol:
        if d[1][4]==1 and d[1][5]==1:
            append_dict(train_lin,d[0],d[1])
        else:
            append_dict(train_ring,d[0],d[1])
for mol in test:
    for d in mol:
        if d[1][4]==1 and d[1][5]==1:
            append_dict(test_lin,d[0],d[1]) 
        else:
            append_dict(test_ring,d[0],d[1])            
for dih in [train_lin,train_ring,test_lin,test_ring]:
    for l in dih.keys():
        dih[l]=np.asarray(dih[l])

In [ ]:
dihedrals_train={}
dihedrals_test={}

for mol in train:
    for d in mol:
        append_dict(dihedrals_train,d[0],d[1])
for mol in test:
    for d in mol:
        append_dict(dihedrals_test,d[0],d[1])

In [ ]:
for dih in [dihedrals_train, dihedrals_test]:
    for l in dih.keys():
        dih[l]=np.asarray(dih[l])

In [ ]:
predictions_lin={}
for dih_key in list(test_lin.keys())[:]:
    if dih_key not in train_lin: continue
    x_train,y_train= train_lin[dih_key][:,:-1],train_lin[dih_key][:,-1]
    x_test,y_test= test_lin[dih_key][:,:-1],test_lin[dih_key][:,-1]
    if (len(y_test)+len(y_train))<100:continue 
    rf=RandomForestRegressor(n_estimators=100,n_jobs=32).fit(x_train, y_train)
    plt.figure(figsize=(8,8))
    y_pred=rf.predict(x_test)
    plt.plot([0,.1],[0,.1],c="r",lw=.5,ls="-.")
    plt.scatter(y_test,y_pred,s=4)
    plt.show()

In [ ]:
predictions_ring={}
for dih_key in list(test_ring.keys())[:]:
    x_test,y_test= test_ring[dih_key][:,:-1],test_ring[dih_key][:,-1]
    if dih_key not in train_ring:continue
    x_train,y_train= train_ring[dih_key][:,:-1],train_ring[dih_key][:,-1]
    if (len(y_test)+len(y_train))<100:continue 
    rf=RandomForestRegressor(n_estimators=100,n_jobs=32).fit(x_train, y_train)
    plt.figure(figsize=(8,8))
    y_pred=rf.predict(x_test)
    plt.scatter(y_test,y_pred,s=4)
    predictions_ring[dih_key]=[(y_test,y_pred)]
    ml,Ml=min(min(y_pred),min(y_test)),max(max(y_pred),max(y_test))
    plt.plot([ml,Ml],[ml,Ml],ls=":",c="k")
    plt.ylabel("PREDICTED")
    plt.xlabel("TRUE")
    plt.show()

# Save Models

In [ ]:
lins={}
rings={}
for mol in mols:
    for d in mol:
        if d[1][4]==1 and d[1][5]==1:
            append_dict(lins,d[0],d[1]) 
        else:
            append_dict(rings,d[0],d[1]) 
for dih in [lins,rings]:
    for l in dih.keys():
        dih[l]=np.asarray(dih[l])

In [ ]:
models_lin={}
for dih_key in list(lins.keys())[:]:
    x_train,y_train= lins[dih_key][:,:-1],lins[dih_key][:,-1]
    rf=RandomForestRegressor(n_estimators=100,n_jobs=32).fit(x_train, y_train)
    rf.n_jobs=1
    models_lin[dih_key]=rf
    print(dih_key)

In [ ]:
models_ring={}
for dih_key in rings:
    x_train,y_train= rings[dih_key][:,:-1],rings[dih_key][:,-1]
    rf=RandomForestRegressor(n_estimators=100,n_jobs=32).fit(x_train, y_train)
    rf.n_jobs=1
    models_ring[dih_key]=rf

In [ ]:
from joblib import dump as jl_dump
from joblib import load as jl_load

In [ ]:
for i in models_lin:
    jl_dump(models_lin[i],"./Saved_Models/Dihedrals/lin_{}{}{}{}.joblib".format(*i))
for i in models_ring:
    jl_dump(models_ring[i],"./Saved_Models/Dihedrals/ring_{}{}{}{}.joblib".format(*i))